library(Seurat)

library(tidyverse)

library(argparse)

library(ggplot2)

library(stringr)

library(dplyr)

parser <- ArgumentParser()

parser$add_argument("--sample",help="sample name", required=TRUE)

parser$add_argument("--rds", help="scRNA rds path", required=TRUE)

parser$add_argument("--VDJ", help='VDJ match_contigs.csv path', required=TRUE)

parser$add_argument("--outdir", help='out dir', required=TRUE)

args <- parser$parse_args()

rds <- readRDS(args$rds)

Idents(rds) <- "orig.ident"

rna <- subset(rds,idents = c(args$sample))

vdj <- read.table(args$VDJ, sep=',', header=T)

df <- rna@meta.data

df$barcode <- rownames(df)

cells <- vdj

barcodes <- unique(cells$barcode)

filter_df <- filter(df, barcode %in% barcodes)

res <- table(filter_df$seurat_clusters)

res <- as.data.frame(res)

# 统计每个cluster中组装出的细胞数
out_csv = stringr::str_glue('{args$outdir}/match_barcodes_celltypes_distribution.txt')

[out_csv](./match_barcodes_celltypes_distribution.txt) 


meta = rna@meta.data

meta$Class = 'NA'

meta[barcodes,'Class'] = 'T/BCR'

meta <- meta %>% drop_na(nCount_RNA)

table(meta$Class == 'T/BCR')

rna@meta.data = meta

rna <- RunUMAP(rna, dims = 1:20)



outP = stringr::str_glue('{args$outdir}/cluster_umap.png')

png(outP, height=1000, width=1000)

UMAPPlot(rna,group.by='seurat_clusters',label=TRUE)

dev.off()

# UMAP-plot of cluster
![Alt text](./cluster_umap.png "optional title")



outP2 = stringr::str_glue("{args$outdir}/assign.png")

png(outP2, height=1000, width=1000)

UMAPPlot(rna,group.by='celltype',label=TRUE,label.box=TRUE)

dev.off()
# UMAP-plot of celltype
![Alt text](./assign.png "optional title")


outP1 = stringr::str_glue("{args$outdir}/umapplot.png")

png(outP1, height=1000, width=1000)

UMAPPlot(rna,group.by='Class',cols=c('grey','red'),label=TRUE)

dev.off()

# UMAP-plot of TCR
![Alt text](./umapplot.png "optional title")


# reference

https://github.com/Chenjunjie1996/Self-Used-Scripts/tree/main/mappingclass